Import SysMLv2 Model:

In [1]:
import syside
import os

MAIN_DIR = "/Users/alejandronietocuatenta/Documents/sysmlv2_omg_introduction/"
# MODEL_DIR = "omg_single_example/"
# MODEL_DIR = "omg_introduction/" # mid complexity example
# MODEL_DIR = "omg_simple_vehicle/" # high complexity example
MODEL_DIR = "project_electronics_basics/"
# MODEL_DIR = "project_clean/"

# Construct full directory path
full_dir = os.path.join(MAIN_DIR, MODEL_DIR)

# Find the first file ending with .sysml
for file in os.listdir(full_dir):
    if file.endswith(".sysml"):
        MODEL_FILE_PATH = os.path.join(full_dir, file)
        break  # stop after finding the first .sysml file

(model, diagnostics) = syside.load_model([MODEL_FILE_PATH])

Export to JSON (minimal):

In [2]:
from pathlib import Path

assert len(model.user_docs) == 1

with model.user_docs[0].lock() as locked:
    json_output = syside.json.dumps(
        locked.root_node, syside.SerializationOptions.minimal()
    )

# Use current working directory (instead of __file__)
output_path = Path.cwd() / "export.json"

# Write JSON file
output_path.write_text(json_output, encoding="utf-8")

print(f"JSON saved to: {output_path}")

JSON saved to: /Users/alejandronietocuatenta/Documents/sysmlv2_omg_introduction/export.json


Create cypher script for neo4j (.cypher):

In [3]:
import json

with open("export.json") as f:
    data = json.load(f)

cypher = []

# 1. Create all nodes
for e in data:
    props = []
    for key in ("@id", "@type", "declaredName", "memberName", "operator", "value"):
        val = e.get(key)
        if val:
            field = "id" if key == "@id" else "type" if key == "@type" else key
            props.append(f'{field}:"{val}"')
    # cypher.append(f'CREATE (:`Node` {{{", ".join(props)}}});') # original
    # cypher.append(f'CREATE ({e.get("@type")}:Node {{{", ".join(props)}}});') # using @type as node label name
    cypher.append(f'CREATE (:{e.get("@type")} {{{", ".join(props)}}});') # using @type as node label

# 2. Create relationships
rels = {
    "owningRelatedElement": "OWNING_RELATED_ELEMENT",
    "ownedRelationship": "OWNED_RELATIONSHIP",
    "memberElement": "MEMBER_ELEMENT",
    "ownedRelatedElement": "OWNED_RELATED_ELEMENT",
    "owningRelationship": "OWNING_RELATIONSHIP",
    "general": "GENERAL",
    "specific": "SPECIFIC",
    "type": "TYPE",
    "typedFeature": "TYPED_FEATURE",
    "importedMembership": "IMPORTED_MEMBERSHIP",
    "memberName": "MEMBER_NAME",
    "memberShortName": "MEMBER_SHORT_NAME",
    "conjugatedType": "CONJUGATED_TYPE",
    "originalType": "ORIGINAL_TYPE"
}
for e in data:
    src_id = e["@id"]
    for k, rel in rels.items():
        if k in e:
            targets = e[k] if isinstance(e[k], list) else [e[k]]
            for t in targets:
                
                # ensure the target is a dict with an @id
                if not isinstance(t, dict) or "@id" not in t:
                    # skip plain strings, numbers, etc.
                    print("t doesn't contain @id objects")
                    continue
                tid = t.get("@id")

                if tid:
                    td_index = next((i for i, item in enumerate(data) if item["@id"] == t.get("@id")), None) # target data index
                    if td_index is None:
                        print(f"td_index is None! {e["@id"]} {e.get("@type")}")
                    else:    
                        cypher.append(
                            # f'MATCH (a:Node {{id:"{src_id}"}}), (b:Node {{id:"{tid}"}}) ' # original
                            f'MATCH (a:{e.get("@type")} {{id:"{src_id}"}}), (b:{data[td_index]['@type']} {{id:"{tid}"}}) ' #
                            f'CREATE (a)-[:{rel}]->(b);'
                        )

with open("export.cypher", "w") as f:
    f.write("\n".join(cypher))


t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
td_index is None! bad9b4c6-a72c-401a-8d57-d3cb2eb00dd3 FeatureTyping
td_index is None! bad9b4c6-a72c-401a-8d57-d3cb2eb00dd3 FeatureTyping
t doesn't contain @id objects
td_index is None! 49e84610-5a06-4bf4-bccb-32783c7e85fd FeatureTyping
td_index is None! 49e84610-5a06-4bf4-bccb-32783c7e85fd FeatureTyping
t doesn't contain @id objects
td_index is None! 64995e14-e823-4cd9-aeda-bab406f295a5 FeatureTyping
td_index is None! 64995e14-e823-4cd9-aeda-bab406f295a5 FeatureTyping
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
t doesn't contain @id objects
td_index is None! ecb1ca31-52fa-4ff1-9f08-5108df76e001 FeatureTyping
td_index is None! ecb1ca31-52fa-4ff1-

Export to JSON (with options) - NOT USED but functional

In [4]:
# from pathlib import Path

# assert len(model.user_docs) == 1

# options = syside.SerializationOptions()  # create an instance
# options = options.with_options(
#     use_standard_names=True,
#     include_derived=True,
#     include_redefined=True,
#     include_default=True,
#     include_optional=True,
#     include_implied=True
# )

# with model.user_docs[0].lock() as locked:
#     json_output = syside.json.dumps(
#         locked.root_node, 
#         options,
#     )

# # Use current working directory (instead of __file__)
# output_path = Path.cwd() / "export.json"

# # Write JSON file
# output_path.write_text(json_output, encoding="utf-8")

# print(f"JSON saved to: {output_path}")